In [4]:
#Import Required Libraries:
import pandas as pd
import networkx as nx
import json

#Load Author Network Data
df = pd.read_csv("https://github.com/umassdgithub/DataViz-Fall2024/blob/main/Week-8-ForceSimulator/data/data_scopus.csv?raw=true")

nodes = df['EID'].values
df=df.fillna(0)
df.head(2)



,Title,Year,EID,Abstract,Publisher,Conference name,Conference date,Authors,Author(s) ID,Authors with affiliations,Source title,Abbreviated Source Title,Cited by
0,Virtual reality applications for the built env...,2020,2-s2.0-85086464158,With its advanced capabilities of immersive an...,Elsevier B.V.,0,0,"Zhang Y., Liu H., Kang S.-C., Al-Hussein M.",57202536919;53984550800;57215426514;6603541102;,"Zhang, Y., Department of Civil and Environment...",Automation in Construction,Autom Constr,0.0
1,"Self-tracking while doing sport: Comfort, moti...",2020,2-s2.0-85082875828,The spread of wearable technologies is paving ...,Academic Press,0,0,"Rapp A., Tirabeni L.",23398572100;57191836654;,"Rapp, A., Computer Science Department, Univers...",International Journal of Human Computer Studies,Int J Hum Comput Stud,0.0


In [5]:
df = pd.read_csv("https://github.com/umassdgithub/DataViz-Fall2024/blob/main/Week-8-ForceSimulator/data/data_scopus.csv?raw=true")
df = df.dropna(subset=['Author(s) ID','Year','Authors','Authors with affiliations'])

# nodes = df['Author(s) ID'].values
df = df.fillna(0)
Graph = nx.Graph()
nodes = {}


for _, row in df.iterrows():
    # Extract authors from the "Authors" column
    authors = row['Authors'].split(',')
    author_Id = row['Author(s) ID'].split(';')
    Title=row['Title']
    Year=row['Year']
    Citations=row['Cited by']
    Publisher=row['Publisher']
    Authorwa=row['Authors with affiliations']

    # Add authors as nodes

    for author in range(len(authors)):

        aid= author_Id[author]
        aname= ';'.join(authors)
        title=Title

        if(aid!=""):
          #print(aid)
         nodes={'id':aid,
          "Authors": aname,
          "Title": title,
          "Year": Year,
          "Citations": Citations,
          "Publisher": Publisher,
          "Author with affiliations":Authorwa
           }
         Graph.add_node(aid,**nodes)






print(Graph)
from networkx.readwrite import json_graph
with open("publication_network.json", "w") as f:
    json.dump(json_graph.node_link_data(Graph), f)



Graph with 1552 nodes and 0 edges


/usr/local/lib/python3.10/dist-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


In [6]:
# Load the data from the JSON file
with open('/content/publication_network.json', 'r') as infile:
    data = json.load(infile)
newnodes=[]
# Create a NetworkX graph
Graph=nx.Graph()

def get_author_country(authors_with_affiliations):
    first_affiliation = authors_with_affiliations.split(';')[0].strip()
    country = first_affiliation.split(',')[-1].strip()
    return country






# Add nodes (publications) to the graph with attributes
for node_data in data['nodes']:
    eid = node_data.get('id')
    authors=node_data.get('Authors')
    title = node_data.get('Title')
    year = node_data.get('Year')
    citations = node_data.get('Citations')
    publisher = node_data.get('Publisher')

    authors_with_affiliations = str(node_data.get('Author with affiliations'))

    country = get_author_country(authors_with_affiliations)

    Graph.add_node(eid,authors=authors, title=title, year=year, citations=citations, publisher=publisher,authors_with_affiliations=authors_with_affiliations, country=country)
# Generate clusters based on authors' countries
clusters = list(nx.connected_components(Graph))

# Assign classes to each cluster
class_mapping = {node: idx for idx, cluster in enumerate(clusters) for node in cluster}
nx.set_node_attributes(Graph, class_mapping, 'class')

# title = data["Title"]
# authors_info = data["Authors with affiliations"].split(';')

# Create a dictionary to store co-authorship relationships
coauthorship_dict = {}
for row in df.iterrows():
    authors = row[1]['Author(s) ID'].split(';')
    for i in range(len(authors)):
      for j in range(i+1, len(authors)):
        if(authors[i]!="" and authors[j]!=""):
          Graph.add_edge(authors[i], authors[j])


print(Graph)



# Save the data as a JSON file with proper character encoding
co_author_data = {'nodes': [{'id': node, **Graph.nodes[node]} for node in Graph.nodes()],'links': [{'source': source, 'target': target} for source, target in Graph.edges()]}

with open('co_author_data.json', 'w', encoding='utf-8') as outfile:
    json.dump(co_author_data, outfile, ensure_ascii=False)

Graph with 1552 nodes and 3049 edges
